# CONVERT FILES WITH DUCK_DB

In [1]:
import pandas as pd
import duckdb
import os
import time

In [2]:
def time_it(func):
    """Decorator to measure the execution time of a function."""
    def wrapper(*args, **kwargs):
        start_time = time.time()  
        result = func(*args, **kwargs) 
        end_time = time.time() 
        elapsed_time = end_time - start_time  
        print(f"Execution time: {elapsed_time:.4f} seconds")  
        return result
    return wrapper

## Ownership history

In [3]:
# path = "E:\dati_moody\ownership_history\links_2022\part-00000-8f9cac6d-cf88-4461-91b4-13c784cdf6a9-c000.snappy.parquet" 

In [4]:
eu27_countries = [
    "AT", "BE", "BG", "HR", "CY", "CZ", 
    "DK", "EE", "FI", "FR", "DE", "GR", 
    "HU", "IE", "IT", "LV", "LT", "LU", 
    "MT", "NL", "PL", "PT", "RO", "SK", 
    "SI", "ES", "SE"
]

In [5]:

TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed\\*.parquet" 
TEMP_TABLE_KEY_FINANCIALS = "..\\data_raw\\key_financials\\key_financials_eur\\*.parquet"


def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")
            
    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            {year} AS year,
            key_financials.number_of_employees AS guo_number_of_employees,
            key_financials.closing_date AS guo_closing_date,
        FROM 
            '{path}' AS main
        LEFT JOIN 
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number
        LEFT JOIN
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_guo
        ON
            main.guo_25 = firmographics_guo.bvd_id_number
        LEFT JOIN (
            SELECT 
                number_of_employees, 
                closing_date,
                bvd_id_number,
                EXTRACT(YEAR FROM closing_date) AS financial_year  
            FROM 
                '{TEMP_TABLE_KEY_FINANCIALS}' 
        ) AS key_financials
        ON 
            main.guo_25 = key_financials.bvd_id_number   -- GHANGE HERE FOR SUBS
        AND 
            EXTRACT(YEAR FROM key_financials.closing_date) = {year}  
        WHERE 
            main."type_of_relation" = 'GUO 25'
        AND 
            main."guo_25" LIKE '{country}%'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df


def convert_to_stata(df, output_path, country, year):
    os.makedirs(output_path, exist_ok=True)
    df.to_stata(f"{output_path}\\{country}_{year}.dta", write_index=False)


@time_it
def fetch_and_convert_to_stata(year, country, path, output_path):
    df = get_ownership_data(year, country, path)
    convert_to_stata(df, output_path, country, year)
    print(f"Data for {country} - {year} has been converted to Stata.")

### Test

In [ ]:
# path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"
country = "AT"
year = "2007"
path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"
output_path = r"..\data_processed\\guo_subs_europee\\guos_eu"

df = get_ownership_data(
    path=path,
    year=year,
    country=country,
)
df

# fetch_and_convert_to_stata(country=country, year=year, path=path, output_path=output_path)

AT - 2007...


,subsidiary_bvd_id,guo_25,subsidiary_nuts2,subsidiary_nace4,guo_nuts2,guo_nace4,guo_type_of_entity,guo_status,year,guo_number_of_employees,guo_closing_date
0,AT9110479845,AT9110277823,AT13 - Wien,6832,AT13 - Wien,8299,Foundation/Research institute,Active,2007,NaN,NaT
1,AT9110281795,AT9110281795,AT13 - Wien,6430,AT13 - Wien,None,Foundation/Research institute,Active,2007,NaN,NaT
2,AT9110387942,AT9110281788,AT13 - Wien,8299,AT13 - Wien,8299,Foundation/Research institute,Active,2007,NaN,NaT
3,AT9110281787,AT9110281787,AT13 - Wien,8299,AT13 - Wien,8299,Foundation/Research institute,Active,2007,NaN,NaT
4,AT9110124882,AT9110281781,AT13 - Wien,7010,AT13 - Wien,8299,Foundation/Research institute,Active,2007,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...
514205,AT9030117226,AT9130116522,AT22 - Steiermark,6120,None,None,None,None,2007,NaN,2007-12-31
514206,AT9050099407,AT9130116522,AT33 - Tirol,None,None,None,None,None,2007,NaN,2007-12-31
514207,AT9050099390,AT9130116522,AT33 - Tirol,4743,None,None,None,None,2007,NaN,2007-12-31
514208,AT9050099407,AT9130116522,AT33 - Tirol,7010,None,None,None,None,2007,NaN,2007-12-31


In [7]:
df.guo_nuts2.isna().sum()

np.int64(351736)

In [ ]:
import time
import json

errors = {}
OUTPUT_PATH = r"..\data_processed\\guo_subs_europee\\guos_eu"

total_start = time.time()

start_time = time.time()
for country in eu27_countries:
    for year in range(2007, 2022):
        year = str(year)
        try:
            path = f"..\\data_raw\\ownership_history\\links_{year}\\*.parquet"
            fetch_and_convert_to_stata(year, country, path, OUTPUT_PATH)      
            end_time = time.time()
            duration = end_time - start_time
            print(f"Data for {country} - {year} has been converted to Stata in {duration:.2f} seconds.")
        except Exception as e:
            errors[f"{country} - {year}"] = str(e)
            print(f"Error {country} - {year}")
            continue


end_time = time.time()
duration = end_time - start_time
total_end = time.time()
total_duration = total_end - total_start
print(f"Total execution time: {total_duration:.2f} seconds.")

with open("errors.json", "w") as f:
    json.dump(errors, f)


AT - 2007...
Data for AT - 2007 has been converted to Stata.
Execution time: 56.4214 seconds
Data for AT - 2007 has been converted to Stata in 56.42 seconds.
AT - 2008...
Data for AT - 2008 has been converted to Stata.
Execution time: 57.1515 seconds
Data for AT - 2008 has been converted to Stata in 57.15 seconds.
AT - 2009...
Data for AT - 2009 has been converted to Stata.
Execution time: 58.2541 seconds
Data for AT - 2009 has been converted to Stata in 58.25 seconds.
AT - 2010...
Data for AT - 2010 has been converted to Stata.
Execution time: 60.8653 seconds
Data for AT - 2010 has been converted to Stata in 60.87 seconds.
AT - 2011...
Data for AT - 2011 has been converted to Stata.
Execution time: 62.7612 seconds
Data for AT - 2011 has been converted to Stata in 62.76 seconds.
AT - 2012...
Data for AT - 2012 has been converted to Stata.
Execution time: 64.3215 seconds
Data for AT - 2012 has been converted to Stata in 64.32 seconds.
AT - 2013...
Data for AT - 2013 has been converted t

In [ ]:
# open errors
with open("errors.json", "r") as f:
    errors = json.load(f)

errors